In [1]:
import skimage
import skimage.io
import skimage.transform

from skimage import measure

import os
import scipy as scp
import scipy.misc

import numpy as np
import logging
import tensorflow as tf
import sys

import aws_fcn16_vgg


In [2]:
ultrasound1 = skimage.io.imread('/Users/gus/CDIPS/nerve-project/train/1_1.tif')

In [3]:
down_x2 = measure.block_reduce(ultrasound1,(2,2),func=np.mean)

In [4]:
down_x2.shape

(210, 290)

In [5]:
img1 = skimage.io.imread("/Users/gus/CDIPS/tensorflow-fcn/test_data/tabby_cat.png")

In [6]:
sess=tf.InteractiveSession()

In [7]:
img1.shape

(368, 489, 3)

In [8]:
v=tf.constant([255.0,0.0,255,0.0])

In [9]:
w=tf.cast(v,tf.uint8)

In [10]:
aws_fcn16_vgg.FC7_SHAPE

[512, 512]

In [10]:
images = tf.placeholder("float")
feed_dict = {images: np.dstack((down_x2,down_x2,down_x2))}
batch_images = tf.expand_dims(images, 0)

In [11]:
vgg_fcn = aws_fcn16_vgg.FCN16VGG(vgg16_npy_path='/Users/gus/CDIPS/vgg16.npy')
with tf.name_scope("content_vgg"):
    vgg_fcn.build(batch_images, random_init_fc8=False,debug=True)


npy file loaded
Layer name: conv1_1
Layer shape: (3, 3, 3, 64)
Layer name: conv1_2
Layer shape: (3, 3, 64, 64)
Layer name: conv2_1
Layer shape: (3, 3, 64, 128)
Layer name: conv2_2
Layer shape: (3, 3, 128, 128)
Layer name: conv3_1
Layer shape: (3, 3, 128, 256)
Layer name: conv3_2
Layer shape: (3, 3, 256, 256)
Layer name: conv3_3
Layer shape: (3, 3, 256, 256)
Layer name: conv4_1
Layer shape: (3, 3, 256, 512)
Layer name: conv4_2
Layer shape: (3, 3, 512, 512)
Layer name: conv4_3
Layer shape: (3, 3, 512, 512)
Layer name: conv5_1
Layer shape: (3, 3, 512, 512)
Layer name: conv5_2
Layer shape: (3, 3, 512, 512)
Layer name: conv5_3
Layer shape: (3, 3, 512, 512)
fc6 Old size: [7, 7, 512, 4096]
fc6 New size: [7, 7, 512, 512]
fc6 x-scaling factor: 1.0
fc6 y-scaling factor: 8.0
Initializer of fc6shape: (7, 7, 512, 512)
(?, ?, ?, 512)
resizing biases
(512,)
fc7 Old size: [1, 1, 4096, 4096]
fc7 New size: [512, 512]
fc7 x-scaling factor: 8.0
fc7 y-scaling factor: 8.0
Initializer of fc7shape: (1, 1, 512

In [12]:
init = tf.initialize_all_variables()
sess.run(init)

In [15]:
vgg_fcn.data_dict['fc6'][0].shape

(25088, 4096)

In [16]:
vgg_fcn.data_dict['fc7'][0].shape

(4096, 4096)

In [17]:
vgg_fcn.data_dict['fc8'][0].shape

(4096, 1000)

In [18]:
vgg_fcn.fc7.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(512)])

In [19]:
25088/512

49.0

In [20]:
vgg_fcn.conv1_1.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(64)])

In [21]:
vgg_fcn.pool1.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(64)])

In [22]:
vgg_fcn.pool2.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(128)])

In [23]:
vgg_fcn.pool3.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(256)])

In [24]:
vgg_fcn.pool4.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(512)])

In [25]:
vgg_fcn.pool5.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(512)])

In [26]:
vgg_fcn.score_fr.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(250)])

In [27]:
vgg_fcn.fuse_pool4.get_shape()

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(2)])

In [13]:
time upscored = sess.run(vgg_fcn.upscore32,feed_dict=feed_dict)

CPU times: user 4.66 s, sys: 472 ms, total: 5.13 s
Wall time: 1.76 s


In [29]:
upscored.shape

(1, 368, 489, 2)

In [30]:
upscored

array([[[[-0.32108346, -0.88854849],
         [-0.34900376, -0.96581358],
         [-0.37692404, -1.04307866],
         ..., 
         [-0.72437876, -0.28433385],
         [-0.67442161, -0.26472464],
         [-0.62446445, -0.2451154 ]],

        [[-0.35885799, -0.9930836 ],
         [-0.39006302, -1.07943869],
         [-0.42126805, -1.16579378],
         ..., 
         [-0.80959976, -0.31778491],
         [-0.75376529, -0.29586869],
         [-0.69793081, -0.27395251]],

        [[-0.39663249, -1.0976187 ],
         [-0.43112227, -1.19306386],
         [-0.46561205, -1.28850889],
         ..., 
         [-0.89482081, -0.35123596],
         [-0.83310902, -0.32701278],
         [-0.77139723, -0.3027896 ]],

        ..., 
        [[ 0.80448097, -0.51931393],
         [ 0.87443584, -0.56447166],
         [ 0.94439071, -0.60962939],
         ..., 
         [-0.11961376,  1.13660085],
         [-0.11136454,  1.05821466],
         [-0.10311531,  0.97982836]],

        [[ 0.72786373, -0.4698

In [27]:
### check what masks we have

In [19]:
dat1 = np.fromfile('/Users/gus/CDIPS/bottleneck_files/fc6pool4mask_batch_1',dtype=np.float32)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/gus/CDIPS/bottleneck_files/fc6pool4mask_batch_1'

In [46]:
dat1[1437696+1916928:2*1916928]

array([ 0.,  1.,  0., ...,  1.,  0.,  1.], dtype=float32)

In [38]:
dat1.shape[0]/4

1916928.0

In [43]:
13*18*4096+26*36*512

1437696